# <font color='red'>Language Models</font>

Here, I will train some <b>n-gram language models</b> on WikiText-2, a corpus of high-quality Wikipedia articles. The dataset was originally introduced in the following paper: https://arxiv.org/pdf/1609.07843v1.pdf. A raw version of the data can easily be viewed here: https://github.com/pytorch/examples/tree/master/word_language_model/data/wikitext-2.

## Preprocessing the Data

To make models more robust, it is necessary to perform some basic preprocessing on the corpora.

* <b>Sentence splitting:</b>&nbsp;&nbsp;&nbsp;&nbsp; We are interested in modeling individual sentences, rather than longer chunks of text such as paragraphs or documents. The WikiTest dataset provides paragraphs; thus, we provide a simple method to identify individual sentences by splitting paragraphs at punctuation tokens (".",  "!",  "?").

* <b>Sentence markers:</b>&nbsp;&nbsp;&nbsp;&nbsp;For both training and testing corpora, each sentence must be surrounded by a start-of-sentence (`<s>`) and end-of-sentence marker (`/s`). These markers will allow models to generate sentences that have realistic beginnings and endings.

* <b>Unknown words:</b>&nbsp;&nbsp;&nbsp;&nbsp;In order to deal with unknown words in the test corpora, all words that do not appear in the vocabulary must be replaced with a special token for unknown words (`<UNK>`) before estimating the models. The WikiText dataset has already done this, and you can read about the method in the paper above. When unknown words are encountered in the test corpus, they should be treated as that special token instead.

In [1]:
!pip install torchdata

ERROR: torchvision 0.7.0+cu101 has requirement torch==1.6.0, but you'll have torch 1.12.1 which is incompatible.
You should consider upgrading via the 'c:\users\wiewi\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.



  Using cached torchdata-0.4.1-cp37-cp37m-win_amd64.whl (1.1 MB)
  Using cached torch-1.12.1-cp37-cp37m-win_amd64.whl (161.9 MB)


In [2]:
START = "<s>"   # Start-of-sentence token
END = "</s>"    # End-of-sentence-token
UNK = "<UNK>"   # Unknown word token

In [5]:
import torchtext
import random
import string
import sys

def preprocess(data, vocab=None):
    final_data = []
    lowercase = string.ascii_lowercase
    
    for paragraph in data:
        paragraph = [x if x != '<unk>' else UNK for x in paragraph.split()]
        
        if vocab is not None:
            paragraph = [x if x in vocab else UNK for x in paragraph]
            
        if paragraph == [] or paragraph.count('=') >= 2: 
            continue
            
        sen = []
        prev_punct, prev_quot = False, False
        
        for word in paragraph:
            if prev_quot:
                if word[0] not in lowercase:
                    final_data.append(sen)
                    sen = []
                    prev_punct, prev_quot = False, False
            if prev_punct:
                if word == '"':
                    prev_punct, prev_quot = False, True
                else:
                    if word[0] not in lowercase:
                        final_data.append(sen)
                        sen = []
                        prev_punct, prev_quot = False, False
                        
            if word in {'.', '?', '!'}: 
                prev_punct = True
            sen += [word]
            
        if sen[-1] not in {'.', '?', '!', '"'}: 
            continue # Prevent a lot of short sentences
        final_data.append(sen)
        
    vocab_was_none = vocab is None
    
    if vocab is None:
        vocab = set()
        
    for i in range(len(final_data)):
        final_data[i] = [START] + final_data[i] + [END]
        
        if vocab_was_none:
            for word in final_data[i]:
                vocab.add(word)
    return final_data, vocab

def getDataset():
    dataset = torchtext.datasets.WikiText2(root='.data', split=('train', 'valid'))
    train_dataset, vocab = preprocess(dataset[0])
    test_dataset, _ = preprocess(dataset[1], vocab)

    return train_dataset, test_dataset

train_dataset, test_dataset = getDataset()

Lets see some examples:

In [15]:
for sentance in train_dataset[7:9]:
    print(sentance)

['<s>', 'A', 'large', 'team', 'of', 'writers', 'handled', 'the', 'script', '.', '</s>']
['<s>', 'The', 'game', "'s", 'opening', 'theme', 'was', 'sung', 'by', 'May', "'n", '.', '</s>']


## The LanguageModel Base Class

I will implement 4 types of language models: a <b>unigram</b> model, a <b>smoothed unigram</b> model, a <b>bigram</b> model, and a <b>smoothed bigram</b> model.

* <b>`__init__(self, trainCorpus)`</b>: Train the language model on `trainCorpus`. This will involve calculating relative frequency estimates according to the type of model implementing.

* <b>`generateSentence(self)`</b>: Return a sentence that is generated by the language model. It should be a list of the form <TT>[&lt;s&gt;, w<sup>(1)</sup>, ..., w<sup>(n)</sup>, &lt;&sol;s&gt;]</TT>, where each <TT>w<sup>(i)</sup></TT> is a word in the vocabulary (including <TT>&lt;UNK&gt;</TT> but exlcuding <TT>&lt;s&gt;</TT> and <TT>&lt;&sol;s&gt;</TT>). I assume that <TT>&lt;s&gt;</TT> starts each sentence (with probability $1$). The following words <TT>w<sup>(1)</sup></TT>, ... , <TT>w<sup>(n)</sup></TT>, <TT>&lt;&sol;s&gt;</TT> are generated according to language model's distribution. The number of words <TT>n</TT> is not fixed; instead, I stop the sentence as soon as I generate the stop token <TT>&lt;&sol;s&gt;</TT>.

* <b>`getSentenceLogProbability(self, sentence)`</b>: Return the <em> logarithm of the probability</em> of <TT>sentence</TT>, which is again a list of the form <TT>[&lt;s&gt;, w<sup>(1)</sup>, ..., w<sup>(n)</sup>, &lt;&sol;s&gt;]</TT>.

* <b>`getCorpusPerplexity(self, testCorpus)`</b>: Compute the perplexity (normalized inverse log probability) of `testCorpus` according to the model. For a corpus $W$ with $N$ words and a bigram model, Jurafsky and Martin tells us to compute perplexity as follows: 

$$Perplexity(W) = \Big [ \prod_{i=1}^N \frac{1}{P(w^{(i)}|w^{(i-1)})} \Big ]^{1/N}$$

In order to avoid underflow, I will need to do all of my calculations in log-space. That is, instead of multiplying probabilities, I add the logarithms of the probabilities and exponentiate the result:

$$\prod_{i=1}^N P(w^{(i)}|w^{(i-1)}) = \exp\Big (\sum_{i=1}^N \log P(w^{(i)}|w^{(i-1)}) \Big ) $$

In [19]:
import math
import random
from collections import defaultdict


class LanguageModel(object):
    def __init__(self, train_corpus):
        '''
        Initialize and train the model 
        '''
        return

    def generate_sentence(self):
        '''
        Generate a sentence by drawing words according to the model's probability distribution.
        '''
        
        raise NotImplementedError()

    def get_sentence_log_probability(self, sentence):
        '''
        Calculate the log probability of the sentence provided. 
        '''

        raise NotImplementedError()
        
    def get_corpus_perplexity(self, test_corpus):
        '''
        Calculate the perplexity of the corpus provided.
        '''

        raise NotImplementedError()

    def print_sentences(self, n):
        '''
        Prints n sentences generated by model.
        '''

        for i in range(n):
            sent = self.generate_sentence()
            prob = self.get_sentence_log_probability(sent)
            print('Log Probability:', prob , '\tSentence:',sent)

---
## <font color='green'>Unigram Model</font>

The probability distribution of a word is given by $\hat P(w)$.

In [40]:
def unigram_counter(dictionary, word):
    if word in dictionary.keys():
        dictionary[word] += 1
    else:
        dictionary[word] = 1

In [46]:
from random import choices
import numpy as np


class UnigramModel(LanguageModel):
    def __init__(self, train_corpus):
        self.train_corpus = train_corpus
        self.P = self.get_tokens_prob()

    def get_tokens_prob(self):
        unigram_count = {}
        
        for sentence in self.train_corpus:
            for word in sentence:
                if word == START:
                    continue
                    
                unigram_counter(unigram_count, word)
        
        N = sum(unigram_count.values())
        return {k: v / N for k, v in unigram_count.items()}
 
    def generate_sentence(self):
        sentence = [START]

        word = ''
        while word != END:
            word = choices(list(self.P.keys()), list(self.P.values()))[0]
            sentence.append(word)

        return sentence

    def get_sentence_log_probability(self, sentence):
        sentence_prob = [self.P[word] for word in sentence[1:]]
        return np.sum(np.log(sentence_prob))
        
    def get_corpus_perplexity(self, test_corpus):
        logP = 0
        N = 0
        
        for sentence in test_corpus:
            N += len(sentence) - 1
            logP += self.get_sentence_log_probability(sentence)
           
        perplexity = math.exp(-logP / N)
        return perplexity 

In [47]:
def run(ModelObject):
    model = ModelObject(train_dataset)
   
    print("--------- 5 sentences from the model ---------")
    model.print_sentences(5)

    print ("\n--------- Corpus Perplexities ---------")
    print ("Training Set:", model.get_corpus_perplexity(train_dataset))
    print ("Testing Set:", model.get_corpus_perplexity(test_dataset))

In [48]:
run(UnigramModel)

--------- 5 sentences from the model ---------
Log Probability: -488.8695654861011 	Sentence: ['<s>', 'they', 'Even', 'group', 'six', '<UNK>', 'present', 'poor', '1912', 'topped', 'Scots', 'the', '@-@', '<UNK>', 'shot', 'also', 'they', 'from', 'commanded', 'high', '(', 'could', 'time', 'war', 'northern', ',', '.', 'She', 'Bracknell', 'overt', 'to', 'behind', 'pitch', 'until', 'about', 'press', '2', ')', 'central', '1933', 'Bob', 'or', 'the', 'Jacob', '<UNK>', ',', 'atmosphere', ',', "'s", 'still', 'his', 'Marines', '1', 'following', 'just', 'the', 'for', '.', 'his', 'to', '.', 'brief', 'there', 'was', 'totals', 'career', 'during', 'the', 'politicians', '</s>']
Log Probability: -137.7611746418094 	Sentence: ['<s>', 'and', 'long', 'war', 'What', 'of', 'sales', 'not', ',', '8', 'the', 'called', 'hydrophobic', 'decided', 'the', '<UNK>', ',', 'of', '.', 'created', 'has', 'runway', '</s>']
Log Probability: -222.0862373636816 	Sentence: ['<s>', 'Sweetums', 'decided', 'M', 'by', '.', 'them', '

---
## <font color='green'>Smoothed Unigram Model</font>

Here, I will implement each of the 4 functions described above for a <b>unigram</b> model with <b>Laplace (add-one) smoothing</b>. The probability distribution of a word is given by $P_L(w)$. This type of smoothing takes away some of the probability mass for observed events and assigns it to unseen events.

If $c(w)$ is the frequency of $w$ in the training data, we can compute $P_L(w)$ as follows:

$$P_L(w)=\frac{c(w)+1}{N+S}$$

In [49]:
class SmoothedUnigramModel(UnigramModel):
    def __init__(self, train_corpus):
        super().__init__(train_corpus)
        self.P = self.get_tokens_prob()
        
    def get_tokens_prob(self):
        unigram_count = {}
        
        for sentence in self.train_corpus:
            for word in sentence:
                if word == START:
                    continue
                    
                unigram_counter(unigram_count, word)
        
        N = sum(unigram_count.values())
        S = len(unigram_count.keys())
        return {k: (v + 1) / (N + S) for k, v in unigram_count.items()}

In [50]:
run(SmoothedUnigramModel)

--------- 5 sentences from the model ---------
Log Probability: -350.63440837925555 	Sentence: ['<s>', 'performed', 'AI', 'neck', 'to', 'without', ',', '.', 'to', 'of', 'Expeditionary', 'the', 'the', 'freeway', 'was', '1990', 'risk', 'of', 'Mississippi', 'and', 'and', 'Kenna', 'BC', ',', 'who', 'the', '.', 'severely', 'schools', 'move', '.', '@-@', 'of', 'head', 'people', 'the', 'DVD', 'for', 'single', 'built', 'reach', 'the', 'perfect', 'until', 'effect', 'out', 'written', 'closed', 'King', 'several', 'future', '</s>']
Log Probability: -128.30731509747412 	Sentence: ['<s>', 'White', 'a', 'she', 'model', 'may', 'This', 'the', 'that', 'Torres', 'with', 'of', 'Age', '<UNK>', 'Other', 'renting', 'a', '49', 'to', '</s>']
Log Probability: -135.5298851522499 	Sentence: ['<s>', '.', 'album', 'forums', 'He', 'run', 'students', 'benign', 'struck', 'and', 'saw', ',', 'the', 'which', 'Paley', 'Battery', '<UNK>', 'prominent', '</s>']
Log Probability: -176.0977621206662 	Sentence: ['<s>', 'to', '"'

---
## <font color='green'>Bigram Model</font>

Here, I will implement each of the 4 functions described above for an <b>unsmoothed bigram</b> model. The probability distribution of a word is given by $\hat P(w'|w)$. Thus, the probability of $w_i$ is conditioned on $w_{i-1}$.

In [63]:
def bigram_counter(dictionary, word1, word2):
    if word1 in dictionary.keys():
        if word2 in dictionary[word1].keys():
            dictionary[word1][word2] += 1
        else:
            dictionary[word1][word2] = 1
    else:
        dictionary[word1] = {word2: 1}
        
def bigram_assigner(dictionary, word1, word2, value):
    if word1 in dictionary.keys():
        if word2 not in dictionary[word1].keys():
            dictionary[word1][word2] = value
    else:
        dictionary[word1] = {word2: value}

In [66]:
class BigramModel(LanguageModel):
    def __init__(self, train_corpus):
        self.train_corpus = train_corpus
        self.P = self.get_tokens_prob()
            
    def get_tokens_prob(self):
        unigram_count = {}
        bigram_count = {}

        for sentence in self.train_corpus:
            for w1, w2 in zip(sentence[:-1], sentence[1:]):
                unigram_counter(unigram_count, w1)
                bigram_counter(bigram_count, w1, w2)
        
        P = {}
        for w1, dic in bigram_count.items():
            for w2, count in dic.items():
                prob = count / unigram_count[w1]
                bigram_assigner(P, w1, w2, prob)
        return P

    def generate_sentence(self):
        sentence = []
        w1 = START
        w2 = None

        while w2 != END:
            w2 = choices(list(self.P[w1].keys()), list(self.P[w1].values()))[0]
            sentence.append(w1)
            w1 = w2
            
        sentence.append(w2)
        return sentence

    def get_sentence_log_probability(self, sentence):
        sentence_sumlogP = 0
        
        for w1, w2 in zip(sentence[:-1], sentence[1:]):
            if w1 in self.P.keys():
                if w2 in self.P[w1].keys():
                    sentence_sumlogP += np.log(self.P[w1][w2])
                    continue
            sentence_sumlogP += np.log(0.0)
                
        return sentence_sumlogP
        
    def get_corpus_perplexity(self, test_corpus):
        logP = 0
        N = 0
        
        for sentence in test_corpus:
            N += len(sentence) - 1
            logP += self.get_sentence_log_probability(sentence)

        perplexity = math.exp(-logP / N)
        return perplexity 

In [67]:
run(BigramModel)

--------- 5 sentences from the model ---------
Log Probability: -51.31658879665059 	Sentence: ['<s>', 'The', 'Early', 'versions', ';', 'its', 'duration', 'meant', 'to', 'be', 'an', 'eye', '.', '</s>']
Log Probability: -71.59756425498249 	Sentence: ['<s>', 'It', 'was', 'involved', 'in', 'North', 'Carolina', 'to', 'open', 'and', 'that', 'the', 'counties', ',', 'and', 'face', '.', '"', '</s>']
Log Probability: -34.579534101736876 	Sentence: ['<s>', 'The', 'New', 'York', 'Zoological', 'Nomenclature', ',', 'it', 'was', 'not', 'use', '.', '</s>']
Log Probability: -70.30705398288661 	Sentence: ['<s>', 'Walker', 'taking', 'the', 'predecessor', ',', 'Denise', '(', '910', 'Arabs', 'of', 'other', 'time', '.', '</s>']
Log Probability: -128.58157795678525 	Sentence: ['<s>', 'Djedkare', "'s", 'residence', 'of', 'the', 'rising', 'to', 'coincide', 'with', 'his', 'head', 'of', 'her', 'that', 'connects', 'to', 'yield', '(', 'Corey', 'Moss', 'said', 'the', 'centenary', 'events', 'in', 'an', 'attack', '.'

c:\users\wiewi\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: divide by zero encountered in log


Testing Set: inf


---
## <font color='green'>Smoothed Bigram Model</font>

Here, I will implement each of the 4 functions described above for a <b>bigram</b> model with <b>absolute discounting</b>. The probability distribution of a word is given by $P_{AD}(w’|w)$.

In order to smooth our model, I need to compute a discounting factor $D$. If $n_k$ is the number of bigrams $w_1w_2$ that appear exactly $k$ times, we can compute $D$ as: 

$$D=\frac{n_1}{n_1+2n_2}$$ 

For each word $w$, I then need to compute the number of bigram types $ww’$ as follows: 

$$S(w)=|\{w’\mid c(ww’)>0\}|$$ 

where $c(ww’)$ is the frequency of $ww’$ in the training data. In other words, $S(w)$ is the number of unique words that follow $w$ at least once in the training data.

Finally, I compute $P_{AD}(w’|w)$ as follows: 

$$P_{AD}(w’|w)=\frac{\max \big (c(ww’)-D,0\big )}{c(w)}+\bigg (\frac{D}{c(w)}\cdot S(w) \cdot P_L(w’)\bigg )$$ 

where $c(w)$ is the frequency of $w$ in the training data and $P_L(w’)$ is the Laplace-smoothed unigram probability of $w’$.

In [84]:
class SmoothedBigramModelAD(BigramModel):
    def __init__(self, train_corpus):
        self.train_corpus = train_corpus
        self.D, self.N, self.S = 0, 0, 0
        self.bigram_count, self.unigram_count = {}, {}
        self.P = self.get_tokens_prob()

    def nk(self, bigram_count, k):
        return sum([1 for w1, dic in bigram_count.items() for w2, count in dic.items() if count == k])
    
                    
    def P_AD(self, w1, w2):
        S_w = len(self.bigram_count[w1].keys())
        c_ww = self.get_bigram_count(w1, w2)
        c_w = self.get_unigram_count(w1)
        P_L = (self.unigram_count[w2] + 1) / (self.N + self.S)
        
        return (max(c_ww - self.D, 0) / c_w) + (self.D / c_w * S_w * P_L)
    
    def get_unigram_count(self, word):
        if word in self.bigram_count.keys():
            return self.unigram_count[word]
        return 0
            
    def get_bigram_count(self, w1, w2):
        if w1 in self.bigram_count.keys():
            if w2 in self.bigram_count[w1].keys():
                return self.bigram_count[w1][w2]
        return 0

    def get_tokens_prob(self):
        for sentence in self.train_corpus:
            unigram_counter(self.unigram_count, sentence[0])
                    
            for w1, w2 in zip(sentence[:-1], sentence[1:]):
                unigram_counter(self.unigram_count, w2)
                bigram_counter(self.bigram_count, w1, w2)
               
        n1, n2 = self.nk(self.bigram_count, 1), self.nk(self.bigram_count, 2)
        self.D = n1 / (n1 + 2 * n2)
        
        self.N = sum([c for w, c in self.unigram_count.items() if w != START])
        self.S = len([w for w, c in self.unigram_count.items() if w != START])
        
        P = {}
        for sentence in self.train_corpus:
            for w1, w2 in zip(sentence[:-1], sentence[1:]):
                prob = self.P_AD(w1, w2)
                bigram_assigner(P, w1, w2, prob)
        return P
    
    def get_sentence_log_probability(self, sentence):
        sentence_sumlogP = 0
        
        for w1, w2 in zip(sentence[:-1], sentence[1:]):
            if w1 in self.P.keys():
                if w2 in self.P[w1].keys():
                    sentence_sumlogP += np.log(self.P[w1][w2])
                    continue
                    
            prob = self.P_AD(w1, w2)
            sentence_sumlogP += np.log(prob)
                
        return sentence_sumlogP

In [85]:
run(SmoothedBigramModelAD)

--------- 5 sentences from the model ---------
Log Probability: -78.30308712471995 	Sentence: ['<s>', 'With', 'the', 'greatest', 'threat', '...', 'I', 'have', 'to', 'Yankovic', 'began', 'using', 'any', 'of', 'them', 'into', 'disgrace', '.', '</s>']
Log Probability: -35.078650995464216 	Sentence: ['<s>', 'He', 'returned', 'to', 'his', 'second', 'season', ',', 'describing', 'it', '.', '</s>']
Log Probability: -87.20450298595823 	Sentence: ['<s>', 'This', 'gave', 'the', '<UNK>', 'back', ',', 'focusing', 'on', 'that', 'it', "'s", 'coffin', ',', 'Dahau', 'and', 'Kauffman', 'themselves', '.', '</s>']
Log Probability: -198.70607196103828 	Sentence: ['<s>', 'By', 'June', ',', 'with', 'the', 'January', '1', 'draw', 'at', 'the', 'cemetery', 'was', 'widespread', ',', 'accidents', 'in', '1987', 'retrospective', 'of', 'Brazil', 'Luiz', '<UNK>', 'in', 'a', 'total', 'of', 'northern', 'China', 'have', 'basis', 'in', 'two', 'greatest', 'hits', ',', 'and', 'non', '@-@', 'down', 'and', 'possessions', '.'